# Part 4: Matching Violations to Google Business Address Information

In [ ]:
# import statements
import pandas as pd
import json
import glob
import re
import time
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

In [ ]:
# view settings
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Test code for reading in the json files

In [ ]:
# # # Test to open a file
# # with open('../data/google_places_results/results_01.json') as fi:
# #     result = json.load(fi)

# # # Alternate approach to reading in the json file
# google_results = pd.read_json(r'../data/google_places_results/results_01.json')
# google_results.head()

In [ ]:
# # Create an empty dataframe with the columns we want, including all of the data from inside of, and outside of, the 'results' blob
# column_names = ['mapped_location', 'address', 'name', 'types', 'address', 'lat', 'long']
# result_addresses = pd.DataFrame(columns = column_names)

# # Go through and open each json results file
# for file in list(glob.glob('../data/google_places_results/*.json')):
#     with open(file) as fi:
#         result = json.load(fi)
# # Write the contents of the 'results' field to a dataframe       
#         google_results = pd.json_normalize(result)
# # Clean up the dataframe columns
#         google_results = google_results.drop(['results'], axis = 1)
#         google_results.columns = ['mapped_location', 'address']
# # Append the contents of each json file to the results dataframe
#         result_addresses = result_addresses.append(google_results)      

### Read in the json files in two parts

One dataframe, `violations_addresses`, should only include the `mapped location` and `address` fields.  
The second dataframe, `google_address`, should only include the fields we want from the `results` field.  
To get only one business name per address, merge the `violations_addresses` and `google_address` fields on the `address` field and run `pd.drop_duplicates()` to get the first business name at the same address.

In [ ]:
# Create an empty dataframe with the columns we want
column_names = ['mapped_location', 'address']
violations_addresses = pd.DataFrame(columns = column_names)

# Go through and open each json results file
for file in list(glob.glob('../data/google_places_results/*.json')):
    with open(file) as fi:
        result = json.load(fi)
# Write the contents of the 'results' field to a dataframe       
        google_results = pd.json_normalize(result)
# Clean up the dataframe columns
        google_results = google_results.drop(['results'], axis = 1)
        google_results.columns = ['mapped_location', 'address']
# Append the contents of each json file to the results dataframe
        violations_addresses = violations_addresses.append(google_results)      

In [ ]:
violations_addresses.head()

In [ ]:
# Check to see how many addresses were passed through to the Google API
violations_addresses.shape

In [ ]:
# Check how many unique values there are
print(violations_addresses.mapped_location.nunique())
print(violations_addresses.address.nunique())
# It looks like some mapped locations have different addresses

In [ ]:
# # Reset the index so that there is a UID for each address/point - might be useful for deduping later
# violations_addresses = violations_addresses.reset_index()
# violations_addresses.head(50)

In [ ]:
# Check the column names
violations_addresses.columns

In [ ]:
# Create an empty dataframe with the columns we want to get the fields we need out of the results blob
column_names = ['name', 'types', 'address', 'lat', 'long']
google_address = pd.DataFrame(columns = column_names)

# Go through and open each json results file
for file in list(glob.glob('../data/google_places_results/*.json')):
    with open(file) as fi:
        result = json.load(fi)
# Write the contents of the 'results' field to a dataframe       
        google_results = pd.json_normalize(result, 'results')
# Clean up the dataframe columns
        google_results = google_results.drop(['business_status', 'icon', 'place_id', 'rating', 'reference', 'scope', 'user_ratings_total', 'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng', 'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng', 'opening_hours.open_now', 'plus_code.compound_code', 'plus_code.global_code', 'photos', 'price_level', 'permanently_closed'], axis = 1)
        google_results.columns = ['name', 'types', 'address', 'lat', 'long']
# Append the contents of each json file to the results dataframe
        google_address = google_address.append(google_results)      

In [ ]:
# Take a look at the dataframe we've created
google_address.head()

In [ ]:
violations_addresses['address'] = violations_addresses['address'].str.replace(r',.+', '')
violations_addresses['address'] = violations_addresses['address'].fillna('')
violations_addresses['address'] = violations_addresses['address'].str.replace('nan', '')
google_address['address'] = google_address['address'].str.replace(r',.+', '')
google_address['address'] = google_address['address'].fillna('')
google_address['address'] = google_address['address'].str.replace('nan', '')

In [ ]:
google_address['address'].unique()

In [ ]:
google_address['address'].nunique()

In [ ]:
google_address.sort_values(by='address')

In [ ]:
google_address = google_address.drop_duplicates(subset='address', keep='first')
google_address.info()

In [ ]:
google_address[google_address['address'].isna() == True]

In [ ]:
# # Join the queried addresses to the Google API results. Keep all of the queried addresses
# result_addresses_joined_01 = violations_addresses.merge(google_address, how = 'left', on = 'address')
# result_addresses_joined_01

In [ ]:
# # Check the columns that were produced
# result_addresses_joined_01.columns

In [ ]:
# # Check how many business names were matched based on address
# result_addresses_joined_01[result_addresses_joined_01['name'].isna() == False]

In [ ]:
# result_addresses_joined_01 = result_addresses_joined_01.drop_duplicates(subset='address', keep='first')
# result_addresses_joined_01

In [ ]:
# # Explode the list in the 'types' column to get a row for each establishment type
# result_addresses_joined_01 = result_addresses_joined_01.explode('types')

In [ ]:
# # Turn those rows into columns
# result_addresses_joined_02 = pd.get_dummies(result_addresses_joined_01['types'], prefix = 'type').reset_index().groupby('index').sum()
# result_addresses_joined_02

### We need to do some data cleanup

If we do a literal string match with light data cleaning (removing city info), we only get matches for 18% (861) of the original 4,698 addresses identified in the hubNashville 311 violations database.  

Let's try [fuzzy string matching](https://medium.com/tim-black/fuzzy-string-matching-at-scale-41ae6ac452c2) instead.

In [ ]:
# A class for matching one list of strings to another
class StringMatch():
    
    def __init__(self, source_names, target_names):
        self.source_names = source_names
        self.target_names = target_names
        self.ct_vect      = None
        self.tfidf_vect   = None
        self.vocab        = None
        self.sprse_mtx    = None
        
        
    def tokenize(self, analyzer='char_wb', n=3):
        '''
        Tokenizes the list of strings, based on the selected analyzer
        :param str analyzer: Type of analyzer ('char_wb', 'word'). Default is trigram
        :param str n: If using n-gram analyzer, the gram length
        '''
        # Create initial count vectorizer & fit it on both lists to get vocab
        self.ct_vect = CountVectorizer(analyzer=analyzer, ngram_range=(n, n))
        self.vocab   = self.ct_vect.fit(self.source_names + self.target_names).vocabulary_
        
        # Create tf-idf vectorizer
        self.tfidf_vect  = TfidfVectorizer(vocabulary=self.vocab, analyzer=analyzer, ngram_range=(n, n))
        
        
    def match(self, ntop=1, lower_bound=0, output_fmt='df'):
        '''
        Main match function. Default settings return only the top candidate for every source string.
        
        :param int ntop: The number of top-n candidates that should be returned
        :param float lower_bound: The lower-bound threshold for keeping a candidate, between 0-1.
                                   Default set to 0, so consider all canidates
        :param str output_fmt: The output format. Either dataframe ('df') or dict ('dict')
        '''
        self._awesome_cossim_top(ntop, lower_bound)
        
        if output_fmt == 'df':
            match_output = self._make_matchdf()
        elif output_fmt == 'dict':
            match_output = self._make_matchdict()
            
        return match_output
        
        
    def _awesome_cossim_top(self, ntop, lower_bound):
        ''' https://gist.github.com/ymwdalex/5c363ddc1af447a9ff0b58ba14828fd6#file-awesome_sparse_dot_top-py '''
        # To CSR Matrix, if needed
        A = self.tfidf_vect.fit_transform(self.source_names).tocsr()
        B = self.tfidf_vect.fit_transform(self.target_names).transpose().tocsr()
        M, _ = A.shape
        _, N = B.shape

        idx_dtype = np.int32

        nnz_max = M * ntop

        indptr = np.zeros(M+1, dtype=idx_dtype)
        indices = np.zeros(nnz_max, dtype=idx_dtype)
        data = np.zeros(nnz_max, dtype=A.dtype)

        ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)

        self.sprse_mtx = csr_matrix((data,indices,indptr), shape=(M,N))
    
    
    def _make_matchdf(self):
        ''' Build dataframe for result return '''
        # CSR matrix -> COO matrix
        cx = self.sprse_mtx.tocoo()

        # COO matrix to list of tuples
        match_list = []
        for row,col,val in zip(cx.row, cx.col, cx.data):
            match_list.append((row, self.source_names[row], col, self.target_names[col], val))

        # List of tuples to dataframe
        colnames = ['violations_addresses_index', 'violations_addresses_address', 'google_address_index', 'google_address_address', 'score']
        match_df = pd.DataFrame(match_list, columns=colnames)

        return match_df

    
    def _make_matchdict(self):
        ''' Build dictionary for result return '''
        # CSR matrix -> COO matrix
        cx = self.sprse_mtx.tocoo()

        # dict value should be tuple of values
        match_dict = {}
        for row,col,val in zip(cx.row, cx.col, cx.data):
            if match_dict.get(row):
                match_dict[row].append((col,val))
            else:
                match_dict[row] = [(col, val)]

In [ ]:
# First, turn the unique values in the violations_addresses and results fields into lists
violations_addresses_list = violations_addresses.address.unique().tolist()
google_addresses_list = google_address.address.unique().tolist()
# Then, apply the block of code above to create a new dataframe which shows the match of addresses
addressmatch = StringMatch(violations_addresses_list, google_addresses_list)
addressmatch.tokenize()
violations_google_address_matches = addressmatch.match()

In [ ]:
# Check out the mapping of violations addresses and google API results
violations_google_address_matches

In [ ]:
violations_google_address_matches.google_address_address.nunique()

In [ ]:
violations_google_address_matches.violations_addresses_address.nunique()

In [ ]:
# Take a look at the Google API results that had multiple matches
multi_match = violations_google_address_matches[violations_google_address_matches.duplicated(subset=['google_address_address']) == True].sort_values(by = 'google_address_index')
multi_match

In [ ]:
# Sort the dataframe based on match score
violations_google_address_matches = violations_google_address_matches.sort_values(by = ['google_address_address', 'score'], ascending = False)
violations_google_address_matches

In [ ]:
# Drop the duplicates, only keep the highest-scoring match
violations_google_address_matches = violations_google_address_matches.drop_duplicates(subset=['google_address_address'],keep = 'first')

In [ ]:
violations_google_address_matches.info()

In [ ]:
violations_google_address_matches.google_address_address.nunique()

_Of the 4,697 addresses passed into the Google API, 3,922 were matched using this fuzzy string matching technique (83.5%). This is *much* better than our simple cleaning + string matching approach. It looks like there are some inexact matches and we could do more work to clean this up, but given the timeframe of this analysis, we feel okay with the results as they stand. Thanks for the suggestion, Tim!_

### Merge it all together

Take the `violations_addresses` and `google_address` dataframes, merge them together based on the mapping dataframe `violations_google_address_matches`, and product a unique list of address-business matches in the `business_violations` dataframe.

In [ ]:
# First, merge on the violations_addresses dataframe
business_violations = violations_addresses.merge(violations_google_address_matches, how = 'left', left_on = 'address', right_on = 'violations_addresses_address')
business_violations.info()

In [ ]:
business_violations.violations_addresses_address.nunique()

In [ ]:
business_violations.head()

In [ ]:
# Next, merge on the google API results dataframe
business_violations = business_violations.merge(google_address, how = 'left', left_on = 'google_address_address', right_on = 'address')
business_violations.info()

In [ ]:
business_violations.drop_duplicates(subset='google_address_address', keep='first')
business_violations.info()

In [ ]:
business_violations.head(50)

In [ ]:
business_violations = business_violations.drop(['address_x', 'violations_addresses_index', 'google_address_index', 'google_address_address', 'address_y'], axis = 1)

In [ ]:
business_violations

In [ ]:
business_violations = business_violations.reset_index()

In [ ]:
business_violations.head()

#### Dummyize the types to make it possible to group violations by type

In [ ]:
business_violation_types = business_violations.explode('types')

In [ ]:
business_violation_types.columns

In [ ]:
# # For some reason, this drops 800+ entries so don't use it
# business_violation_types = pd.concat([business_violation_types, pd.get_dummies(business_violation_types.types)], 1).groupby(['mapped_location', 'violations_addresses_address', 'score', 'name', 'lat', 'long']).sum().reset_index()

In [ ]:
business_violation_types

In [ ]:
business_violation_types = pd.get_dummies(business_violation_types['types'], prefix = 'type').reset_index().groupby('index').sum()

In [ ]:
business_violation_types.info()

In [ ]:
business_violation_types.columns

In [ ]:
# Turn the index into a column
business_violation_types.reset_index(inplace=True)

In [ ]:
business_violation_types.info()

In [ ]:
business_violation_types.index.nunique()

In [ ]:
# This is the full dataset with dummyized types
business_violations_with_types = business_violations.merge(business_violation_types, how = 'outer', on = 'index')
business_violations_with_types.head()

In [ ]:
# What are the business without any data?
business_violations_with_types[business_violations_with_types['types'].isna() == True]

In [ ]:
business_violations_with_types.columns = ['index', 'mapped_location', 'address', 'match_score', 
                                          'business_name', 'business_types', 'lat', 'long', 'type_accounting', 'type_amusement_park', 
                                          'type_aquarium', 'type_art_gallery', 'type_atm', 'type_bakery', 'type_bank', 
                                          'type_bar', 'type_beauty_salon', 'type_bicycle_store', 'type_book_store', 
                                          'type_bowling_alley', 'type_cafe', 'type_car_dealer', 'type_car_rental', 
                                          'type_car_repair', 'type_car_wash', 'type_cemetery', 'type_church', 
                                          'type_city_hall', 'type_clothing_store', 'type_convenience_store', 
                                          'type_courthouse', 'type_dentist', 'type_department_store', 'type_doctor', 
                                          'type_drugstore', 'type_electrician', 'type_electronics_store', 
                                          'type_establishment', 'type_finance', 'type_florist', 'type_food', 
                                          'type_funeral_home', 'type_furniture_store', 'type_gas_station', 
                                          'type_general_contractor', 'type_grocery_or_supermarket', 'type_gym', 
                                          'type_hair_care', 'type_hardware_store', 'type_health', 'type_home_goods_store', 
                                          'type_hospital', 'type_insurance_agency', 'type_jewelry_store', 'type_laundry', 
                                          'type_lawyer', 'type_library', 'type_liquor_store', 'type_local_government_office', 
                                          'type_locksmith', 'type_lodging', 'type_meal_delivery', 'type_meal_takeaway', 
                                          'type_movie_rental', 'type_movie_theater', 'type_moving_company', 'type_museum', 
                                          'type_night_club', 'type_painter', 'type_park', 'type_parking', 'type_pet_store', 
                                          'type_pharmacy', 'type_physiotherapist', 'type_place_of_worship', 'type_plumber', 
                                          'type_point_of_interest', 'type_police', 'type_post_office', 'type_premise', 
                                          'type_primary_school', 'type_real_estate_agency', 'type_restaurant', 
                                          'type_roofing_contractor', 'type_rv_park', 'type_school', 'type_secondary_school', 
                                          'type_shoe_store', 'type_shopping_mall', 'type_spa', 'type_stadium', 'type_storage', 
                                          'type_store', 'type_supermarket', 'type_synagogue', 'type_tourist_attraction', 
                                          'type_travel_agency', 'type_university', 'type_veterinary_care', 'type_zoo']

In [ ]:
business_violations_with_types.info()

In [ ]:
business_violations_with_types.address.nunique()

In [ ]:
business_violations_with_types_for_merge = business_violations_with_types.drop(['index', 'mapped_location', 'match_score', 'lat', 'long'], axis = 1)

### Read in the violations data and join it up to get business information

In [ ]:
# Read in the COVID-19 violation reports dataset
violations = pd.read_csv('../data/covid_violations.csv')
violations.head()

In [ ]:
violations.info()

In [ ]:
violations_with_business_info = violations.merge(business_violations_with_types_for_merge, how = 'left', on = 'address')
violations_with_business_info

In [ ]:
# Take a look at the violations results that had multiple matches
multi_match_violations = violations_with_business_info[violations_with_business_info.duplicated(subset=['request_no']) == True].sort_values(by = 'request_no')
multi_match_violations.head(100)

In [ ]:
# How many unique request ids are getting duplicated?
multi_match_violations.request_no.nunique()

In [ ]:
# Since the problem seems to be mostly with NaN values, do a quick dedupe
violations_with_business_info = violations_with_business_info.drop_duplicates(subset=['request_no', 'address'],keep = 'first')

In [ ]:
violations_with_business_info.info()

In [ ]:
violations_with_business_info.head(50)

In [ ]:
print(violations_with_business_info.columns.tolist())

In [ ]:
# Write it out to a CSV
violations_with_business_info.to_csv('../data/violations_with_business_info.csv')